# Assignment 5: Classification

## Step 1. Assumptions

* $N$ is the number of synapses (observations)
* According to Z-coordinate, divide the data into 11 slices.  
* Assume the mean density of synapses in a slice can be classified into 2 groups $W_i$, where $W_i = \{\text{High density},\text{ }\text{Low density}\}$  



## Step 2. Define classification problem

Hence, we want to use the position of $X$ and $Y$ and to pridict the density

* $H_0 = N \perp\!\!\!\perp X, Y$ positions
* $H_1 = N \text{ is not} \perp\!\!\!\perp X, Y$ positions

$X_i = X\text{ position} $  
$Y_i = Y\text{ position} $  

The objective is to minimize the expected error:  
$E[l] = \sum_{n=1}^{\infty}I(\hat{W_i} \neq W_i)$, where $I$ is the indicator function.


## Step 3. Provide algorithms

Classification methods:



## Step 4. Sample data from null and alternative model

## Step 5. Compute accuracy

## Step 6. Apply on real data

## Step 7. Interpret